In [ ]:
import imagej
from pathlib import Path
import os

import multiscale.LINK_system.coordinate as coord
import multiscale.ultrasound.reconstruction as recon
import multiscale.itk.transform as tran
import multiscale.itk.registration as reg
import multiscale.itk.itk_plotting as iplt

# Stitch ultrasound using ImageJ

In [ ]:
data_dir = Path(Path.home(), r'Box\PyImageJ paper')
ij_path = str(Path(data_dir, 'Fiji.app'))
ij = imagej.init(ij_path, headless=False)

In [ ]:
# Temporary cell until we update ij with window
from jnius import autoclass
WindowManager = autoclass('ij.WindowManager')
ij.window = WindowManager

Define the location of files needed for the analysis

In [ ]:
mat_dir = Path(data_dir, r'L38 Fiducial Z13115\Run-1')
intermediate_save_dir = Path(data_dir, r'L38 Fiducial Z13115\Run-1 Intermediate')
output_dir = data_dir
pl_path = Path(data_dir, 'L38 Fiducial Z13115.pos')

Specify arguments for the stitching

In [ ]:
fuse_args = {'downsampling': 3, 'fused_image': '[Display using ImageJ]'}
search_str = 'IQ.mat'
output_name = 'L38 fiducial Z13115.tif'
overwrite_dataset = True
overwrite_tif = True

For demonstration purposes, we are sending the stitched image into ImageJ and then extracting the result to numpy

In [ ]:
assembler = recon.UltrasoundImageAssembler(mat_dir, output_dir, ij, pl_path=pl_path, 
                                           intermediate_save_dir=intermediate_save_dir, 
                                           fuse_args=fuse_args, search_str=search_str, output_name=output_name,
                                           overwrite_dataset=overwrite_dataset, overwrite_tif=overwrite_tif)
assembler.assemble_bmode_image()

Convert the ImageJ1 image into a numpy array

In [ ]:
us_ij = ij.window.getCurrentImage()
us_array = ij.py.from_java(us_ij)

In [ ]:
        def to_window(array, name=None):
            """
            Converts the numpy array into an image window with the specified name
            """
            if name is None:
                ij.ui().show(array)
            else:
                ij.ui().show(name, ij.py.to_java(array))
        ij.window.to_window = to_window

In [ ]:
ij.window.to_window

In [ ]:
ij.window.to_window(ij.py.to_java(us_array))

Get the voxel size of the ultrasound image

In [ ]:
axialRes = us_ij.getCalibration().pixelHeight
lateralRes = us_ij.getCalibration().pixelWidth
elevationRes = us_ij.getCalibration().pixelDepth
us_spacing = [lateralRes, elevationRes, axialRes]

Close the ImageJ1 windows as they are no longer necessary

In [ ]:
ij.window.closeAllWindows()

# Register ultrasound and microscopy using SimpleITK

## Image opening
Define and open the image using the multiscale-imaging python package.  This results in two SimpleITK images that are in separate coordinate systems.

In [ ]:
# This opens a microscopy image stitched using the Grid/Collection stitching algorith
mpm_path = Path(data_dir, r'L38 Fiducial MPM Down4x.tif')

# This points to a file that contains the origins of the stitched image
mpm_origin_path = Path(data_dir, r'L38 Fiducial MPM settings.ome.tif')

# This is an externally measured variable that comes from an indicator gauge and gives the Z/Axial coordinate location
us_height = 13115

# This points to the settings of the US acquisition for coordinate definition
us_param_path = Path(mat_dir, r'L38 fiducial Z' + str(us_height) + '_Run-1_Settings.mat')

# This specifies what dynamic range to open the US image in as dB
dynamic_range = 50

In [ ]:
mpm_image = coord.open_microscopy(mpm_path, mpm_origin_path, downsample_factor=4)

In [ ]:
us_image = coord.convert_array_to_us(us_array, pl_path, us_param_path, us_spacing, us_height, dynamic_range)

## Acquire points
Use the manual landmark specification to get the points.

In [ ]:
point_interface = iplt.RegistrationPointDataAcquisition(mpm_image, us_image)

In [ ]:
transform = tran.landmarks_to_translation(point_interface.get_points_flat())

In [ ]:
final_transform, metric, stop = reg.register(mpm_image, us_image, initial_transform=initial_transform)

In [ ]:
iplt.plot_overlay(mpm_image, us_image, final_transform, downsample=False)